Jupyter Notebook for AFNI preprocessing and pRF mapping of the primary visual cortex using public Human Connectome Project Data (Benson et al., 2018)

The Human Connectome Data set is organised into:

The data is converted into BIDS-like organisation with pseudo-BIDS naming (no json sidecars):

Script logging is handled by execution command (output pipelined into output.~): - not needed in jupyter notebook?
    bash -e hcp_preproc.sh 2>&1 | tee hcp_preproc.log

In [3]:
# Directory specifications
baseDir=~/projects/hcp_retinotopy
subjects=("sub-01")

# Preprocessing parameters
overwrite=false # Whether to overwrite existing outputs

epiDrop=0 # Number of initial volumes to drop
smoothingFWHM=2.0 # Smoothing kernel in mm
highPassFilter=0.01 # High-pass filter cutoff in Hz
motionThreshold=0.5 # Motion threshold in mmP

# pRF mapping parameters
modelType="2D Gaussian" # Type of pRF model
gridSize=50 # Size of the grid for model fitting
numIterations=1000 # Number of iterations for model fitting

Setup the script variables and docker help function

In [4]:
# Set sub for convenience
sub=${subjects[0]}

# source and derivatives directory
sorDir=${baseDir}/sourcedata/${sub}
outDir=${baseDir}/derivatives/afni-preproc/${sub}
sorDocDir=/home/afni_user/work/sourcedata/${sub}
outDocDir=/home/afni_user/work/derivatives/afni-preproc/${sub}

# verify that the results directory does not yet exist
if [ -d "$outDir" ] && [ "$overwrite" = "true" ]; then
    echo "Output directory '$outDir' already exists and overwrite protection is set '$overwrite'"
elif [ ! -d "$outDir" ]; then
    mkdir -p "${outDir}/func"
    mkdir -p "${outDir}/anat"
    mkdir -p "${outDir}/fmap"
fi

# Start afni docker container without terminal attachment
if [ "$(docker ps -q -f name=afni)" ]; then
    echo "afni container is running"
elif [ "$(docker ps -aq -f name=afni)" ]; then
    echo "Starting existing afni container"
    docker start afni
else
    echo "Creating and starting AFNI docker container"
    docker run -d --name afni -v ${baseDir}:/home/afni_user/work afni/afni_make_build sleep infinity
fi


Creating and starting AFNI docker container
5991c3b5f7b568c5721375cf4796fedca0c17ff401522ffe2c9b8cb46149bb0a


In [5]:
# Check sourcedata for data, sorted by unique functional tasks, fmap blip directions, and anatomical images

# The logic of this approach is flawed and a poor execution to deal with multiple runs of the same task/acquisition, like an SBRef alongside the main run, where both share the same task/acq label. 
# It feels redundant to have to run two loops to first identify unique tasks/acquisitions and then loop through them again to extract the relevant info.
# Additionally, the task extraction regex is repeated in both task and SBref loops with an exclusion condition to prevent SBrefs from being added to the task list and vice versa.
# Maybe using associative arrays to store unique tasks/acquisitions as keys?? Using a BIDs json manifest to list expected files and register their relationships would be the best approach!!
# p.s. cell output is unnecessarily verbose, return what was found in the file tree to aid quick visual identification

tasks=()
sbrefs=()
for bold in ${sorDir}/func/${sub}_task-*; do
    if [ ! -e "$bold" ]; then
        continue
    fi
    # Skip SBref files for now
    if [[ "$bold" == *"-SBref"* ]]; then
        continue
    fi
    # extract task-<value> pair value
    boldTask=$(echo "$bold" | sed -E 's/.*task-([^_]+).*/\1/')
    tasks+=("$boldTask")
done
unset boldTask

# Sort through each unique task
i=0
n=${#tasks[@]}
while [ $i -lt $n ]; do
    task=${tasks[$i]}
    echo "Found bold task ${task}"
    # Exclude SBRef files from main task wildcard using negation
    docker exec afni sh -c "for f in ${sorDocDir}/func/${sub}_task-${task}*; do [[ \"\$f\" != *\"SBref\"* ]] && 3dinfo -verb \"\$f\"; done"
    docker exec afni sh -c "3dinfo -verb ${sorDocDir}/func/${sub}_task-${task}*-SBref*"
    tasks[$i]=$(docker exec afni sh -c "for f in ${sorDocDir}/func/${sub}_task-${task}*; do [[ \"\$f\" != *\"SBref\"* ]] && 3dinfo -prefix_noext \"\$f\"; done" | xargs)
    sbrefs[$i]=$(docker exec afni sh -c "3dinfo -prefix_noext ${sorDocDir}/func/${sub}_task-${task}*-SBref*" | xargs)
    echo "Acquisition ${task} set to ${tasks[$i]}"
    ((i++))
done
unset task sbref i n

# # Locate SBRef images
# sbrefs=()
# for ref in ${sorDir}/fmap/${sub}_task-*; do
#     if [ ! -e "$ref" ]; then
#         continue
#     fi
#     if [[ "$ref" == *"sbref"* ]]; then
#         continue
#     fi
#     refImg=$(echo "$ref" | sed -E 's/.*task-([^_]+).*/\1/')
#     sbrefs+=("$refImg")
# done
# unset refImg
# # Sort through each unique SBRef
# i=0
# n=${#blips[@]}
# while [ $i -lt $n ]; do
#     blip=${blips[$i]}
#     echo "Found blip direction ${blip}"
#     docker exec afni sh -c "3dinfo -verb ${sorDocDir}/fmap/${sub}_dir-${blip}*"
#     blips[$i]=$(docker exec afni sh -c "3dinfo -prefix_noext ${sorDocDir}/fmap/${sub}_dir-${blip}*")
#     echo "Acquisition ${blip} set to ${blips[$i]}"
#     ((i++))
# done
# unset blip i n

# Locate field map blip images
blips=()
for epi in ${sorDir}/fmap/${sub}_dir-*; do
    if [ ! -e "$epi" ]; then
        continue
    fi
    blipDir=$(echo "$epi" | sed -E 's/.*dir-([^_]+).*/\1/')
    blips+=("$blipDir")
done
unset blipDir
# Sort through each unique blip direction
i=0
n=${#blips[@]}
while [ $i -lt $n ]; do
    blip=${blips[$i]}
    echo "Found blip direction ${blip}"
    docker exec afni sh -c "3dinfo -verb ${sorDocDir}/fmap/${sub}_dir-${blip}*"
    blips[$i]=$(docker exec afni sh -c "3dinfo -prefix_noext ${sorDocDir}/fmap/${sub}_dir-${blip}*" | xargs)
    echo "Acquisition ${blip} set to ${blips[$i]}"
    ((i++))
done
unset blip i n

# Locate anatomical images
strucs=()
for img in ${sorDir}/anat/${sub}_acq-*; do
    if [ ! -e "$img" ]; then
        continue
    fi
    strucImg=$(echo "$img" | sed -E 's/.*acq-([^_]+).*/\1/')
    strucs+=("$strucImg")
done
unset strucImg
# Sort through each unique anatomical acquisition
i=0
n=${#strucs[@]}
while [ $i -lt $n ]; do
    struc=${strucs[$i]}
    echo "Found anatomical acquisition ${struc}"
    docker exec afni sh -c "3dinfo -verb ${sorDocDir}/anat/${sub}_acq-${struc}*"
    strucs[$i]=$(docker exec afni sh -c "3dinfo -prefix_noext ${sorDocDir}/anat/${sub}_acq-${struc}*" | xargs)
    echo "Acquisition ${struc} set to ${strucs[$i]}"
    ((i++))
done
unset struc i n

echo "Summary of data found for subject ${sub}:"
for acq in "${tasks[@]}" "${sbrefs[@]}" "${blips[@]}" "${strucs[@]}"; do
    echo "$acq"
done
echo "If this is not correct, please check the sourcedata directory and naming conventions"

Found bold task RETBAR1
++ 3dinfo: AFNI version=AFNI_25.2.17 (Sep 26 2025) [64-bit]

Dataset File:    /home/afni_user/work/sourcedata/sub-01/func/sub-01_task-RETBAR1_dir-AP.nii.gz
Identifier Code: XYZ_hiLmdXKu5N66NtWZu-meUg  Creation Date: Thu Oct 30 22:11:58 2025
Template Space:  ORIG
Dataset Type:    Echo Planar (-epan)
Byte Order:      LSB_FIRST {assumed} [this CPU native = LSB_FIRST]
Storage Mode:    NIFTI
Storage Space:   861,900,000 (862 million) bytes
Geometry String: "MATRIX(1.599213,-0.035087,-0.035848,-95.73888,-0.015795,-1.437512,0.70236,37.14211,0.04761,0.701661,1.437152,-111.5008):130,130,85"
Data Axes Tilt:  Oblique (26.075 deg. from plumb)
Data Axes Approximate Orientation:
  first  (x) = Right-to-Left
  second (y) = Posterior-to-Anterior
  third  (z) = Inferior-to-Superior   [-orient RPI]
R-to-L extent:   -95.739 [R] -to-   110.661 [L] -step-     1.600 mm [130 voxels]
A-to-P extent:  -169.258 [A] -to-    37.142 [P] -step-     1.600 mm [130 voxels]
I-to-S extent:  -111.5

In [6]:
echo ${blips[@]}
echo ${tasks[@]}
echo ${sbrefs[@]}
echo ${strucs[@]}


sub-01_dir-AP_epi sub-01_dir-PA_epi
sub-01_task-RETBAR1_dir-AP sub-01_task-RETBAR2_dir-PA sub-01_task-RETCCW_dir-AP sub-01_task-RETCON_dir-PA sub-01_task-RETCW_dir-PA sub-01_task-RETEXP_dir-AP
sub-01_task-RETBAR1_dir-AP_acq-SBref sub-01_task-RETBAR2_dir-PA_acq-SBref sub-01_task-RETCCW_dir-AP_acq-SBref sub-01_task-RETCON_dir-PA_acq-SBref sub-01_task-RETCW_dir-PA_acq-SBref sub-01_task-RETEXP_dir-AP_acq-SBref
sub-01_acq-MPR1_T1w sub-01_acq-MPR2_T1w sub-01_acq-SPC1_T2w


EPI spatial distortion correction (from susceptibility induced off-resonance fields) using reverse phase encoded acquisitions. Fmap correction of b0 field inhomogenaties using blip forwards and reverse echo planar images, warp adjustment maps are calculated using a meet in the middle warp between the epi pair. 3dQwarp
              NB: in stdev calc, input is detrended by removing mean+slope

In [7]:
echo "Creating blip summary statistics images blip_stats - subbrik median, mean, stdev"
for blip in "${blips[@]}"; do
    echo "Processing blip direction ${blip}"
    echo ">${blip}<"
    echo "prefix ${outDocDir}/fmap/${blip}_stats"
    echo "input ${sorDocDir}/fmap/${blip}*"
    docker exec afni sh -c "3dTstat -median -mean -stdev -prefix ${outDocDir}/fmap/${blip}_stats ${sorDocDir}/fmap/${blip}*"
done
#docker exec afni sh -c "3dTstat -median -prefix ${derDocDir}/fmap/${sub}_dir-PA_median ${sorDocDir}/fmap/${sub}_dir-PA*"

Creating blip summary statistics images blip_stats - subbrik median, mean, stdev
Processing blip direction sub-01_dir-AP_epi
>sub-01_dir-AP_epi<
prefix /home/afni_user/work/derivatives/afni-preproc/sub-01/fmap/sub-01_dir-AP_epi_stats
input /home/afni_user/work/sourcedata/sub-01/fmap/sub-01_dir-AP_epi*
++ 3dTstat: AFNI version=AFNI_25.2.17 (Sep 26 2025) [64-bit]
++ Authored by: KR Hammett & RW Cox
*+ WARNING:   If you are performing spatial transformations on an oblique dset,
  such as /home/afni_user/work/sourcedata/sub-01/fmap/sub-01_dir-AP_epi.nii.gz,
  or viewing/combining it with volumes of differing obliquity,
  you should consider running: 
     3dWarp -deoblique 
  on this and  other oblique datasets in the same session.
 See 3dWarp -help for details.
++ Oblique dataset:/home/afni_user/work/sourcedata/sub-01/fmap/sub-01_dir-AP_epi.nii.gz is 26.074936 degrees from plumb.
** ERROR: output dataset name 'sub-01_dir-AP_epi_stats' conflicts with existing file
** ERROR: dataset NOT wri

Caclulate warp displacement using the opposite distortions of the blips. The datasets are assumed to be well aligined.

In [ ]:
docker exec afni sh -c "3dQwarp -plusminus -pmNames AP PA "